In [1]:
import os
import json
import glob

import numpy as np
import cv2 as cv

In [ ]:
path = "/home/rob/Git/meta-fsl-nas/data/OmniPrint-metaX"
dest_base_path = "/home/rob/Git/meta-fsl-nas/data/OmniPrint"
subsets = ['meta1', 'meta2', 'meta3', 'meta4', 'meta5']

meta1_labels = []
meta2_labels = []
meta3_labels = []
meta4_labels = []
meta5_labels = []


for subset, labels in zip(subsets, [meta1_labels, meta2_labels, meta3_labels, meta4_labels, meta5_labels]):
    dest_dataset_path = f"{dest_base_path}/{subset}"
    dataset_path = os.path.join(path, subset)

    for char_path in list(glob.iglob(f'{dataset_path}/*')):
        # Divide into sections per class of 20 samples
        n_samples = len(list(glob.iglob(f'{char_path}/data/*.png')))
        if n_samples % 20 != 0:
            print(f"Size of {path} contains more than 20 samples per class")

        char_type = os.path.split(char_path)[-1]

        prefix_idx = 0
        for i in range(n_samples):
            # Create initial directory
            if i == 0:
                os.makedirs(f"{dest_dataset_path}/{char_type}/character_{prefix_idx}/")

            # Read image
            sample_path = f"{char_path}/data/{char_type}_{i}.png"
#             print(sample_path)
            img = cv.imread(sample_path)

            # Write to destination path
            dest_path = f"{dest_dataset_path}/{char_type}/character_{prefix_idx}/character_{i}.png"
            cv2.imwrite(dest_path, img)

            if (i+1) % 20 == 0 and i != 0:
                # Track labels
                label_ar = [subset, char_type, f"character_{prefix_idx}"]
                labels.append(label_ar)
                
                prefix_idx += 1
                if (i+1) != n_samples:
                    os.makedirs(f"{dest_dataset_path}/{char_type}/character_{prefix_idx}/")

In [129]:
seed = 1
with open("/home/rob/Git/meta-fsl-nas/data/OmniPrint/meta1_labels.json", 'w') as f:
    json.dump(meta1_labels, f)
split_train_val_test(meta1_labels, seed, meta_set="meta1")

with open("/home/rob/Git/meta-fsl-nas/data/OmniPrint/meta2_labels.json", 'w') as f:
    json.dump(meta2_labels, f)
split_train_val_test(meta2_labels, seed, meta_set="meta2")
    
with open("/home/rob/Git/meta-fsl-nas/data/OmniPrint/meta3_labels.json", 'w') as f:
    json.dump(meta3_labels, f)
split_train_val_test(meta3_labels, seed, meta_set="meta3")
    
with open("/home/rob/Git/meta-fsl-nas/data/OmniPrint/meta4_labels.json", 'w') as f:
    json.dump(meta4_labels, f)
split_train_val_test(meta4_labels, seed, meta_set="meta4")
    
with open("/home/rob/Git/meta-fsl-nas/data/OmniPrint/meta5_labels.json", 'w') as f:
    json.dump(meta5_labels, f)
split_train_val_test(meta5_labels, seed, meta_set="meta5")

In [128]:
def split_train_val_test(labels, seed, meta_set):
    np.random.seed(2)
    np.random.shuffle(labels)
    
    train, val, test = labels[:900], labels[900:1049], labels[1049:]
    
    with open(f"/home/rob/Git/meta-fsl-nas/data/OmniPrint/{meta_set}_train_labels.json", 'w') as f:
        json.dump(train, f)
        
    with open(f"/home/rob/Git/meta-fsl-nas/data/OmniPrint/{meta_set}_val_labels.json", 'w') as f:
        json.dump(val, f)
        
    with open(f"/home/rob/Git/meta-fsl-nas/data/OmniPrint/{meta_set}_test_labels.json", 'w') as f:
        json.dump(test, f)